In [1]:
#| warning: false
#| code-fold: true
#import library
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import geopandas as gpd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

#render ploty
pio.renderers.default = "notebook"

#import data
us_states=gpd.read_file("../final_data/tl_2020_us_state.shp")
df = pd.read_csv("../final_data/final_data.csv")

# Initialize the Dash app
app = dash.Dash(__name__)

#normalize data
columns_to_normalize = ['personal.income', 'Median.Rent', 'median.sale.price',
                        'Walkable', 'Unemployment', 'Purchase_power_100_dollars',
                        'Violent_Crime_Rate_per_100k', 'Non_Violent_Crime_Rate_per_100k']

scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Columns where lower values are better
columns_lower_better = ['Median.Rent', 'Unemployment', 'Violent_Crime_Rate_per_100k', 'Non_Violent_Crime_Rate_per_100k']

# Invert scaling for those columns
df[columns_lower_better] = 1 - df[columns_lower_better]

# For clarity sake
df['Political_Affiliation'] = df['Political_Affiliation'].replace({'D': 'Democrat', 'R': 'Republican', 'I': 'Independent'})

## Introduction

Have you ever dreamed of living in a big city?

Or maybe you already live in one and you are looking for a new place to move to?

Do you feel like you are missing out on something?

Is there something more to life than what you are currently experiencing?

If you answered yes to any of these questions, then you've come to the right place!

This website is dedicated to helping people like you find the perfect city to live in. Whether you are an aspiring artist, a young professional, a graduate student, or a retiree, we have the perfect city for you. Our website is designed to help you find the perfect city based on essential living factors such as cost of living, quality of life, safety, and more. We have taken the most prominent metropolitan areas in the United States and provided their livability based on your personal preferences at your fingertips; so you can make an informed decision about where to live. All you have to do is adjust the sliders to your preferences and we will do the rest. 

The bigger the circle, the better the city is for you! Additionally, we have provided a complementary table that ranks the cities based on your preferences.

**So what are you waiting for? Find your next adventure now!**

In [2]:
#| warning: false
#| code-fold: true
# Define the layout of the app
app.layout = html.Div([
    html.H1("Choosing Where to Live",style={"color": "black"}),
    html.Div([
        html.Label("Importance of Rent",style={"color": "black"}),
        dcc.Slider(
            id="Median Rent",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Income",style={"color": "black"}),
        dcc.Slider(
            id="Personal Income",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Home Sale Price",style={"color": "black"}),
        dcc.Slider(
            id="Median Sale Price",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Walkablity",style={"color": "black"}),
        dcc.Slider(
            id="Walkable",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Job Security and Opportunity",style={"color": "black"}),
        dcc.Slider(
            id="Unemployment",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Purchasing Power",style={"color": "black"}),
        dcc.Slider(
            id="Purchasing Power",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Importance of Violent Crime Rate",style={"color": "black"}),
        dcc.Slider(
            id="Crime Rate",
            min=0,
            max=1,
            step=0.1,
            value=0.5,
            marks={i/10: str(i/10) for i in range(11)},
            tooltip={"always_visible": False, "placement": "bottom"}
        ),
        html.Label("Variable to Visualize", style={"color": "black"}),
        dcc.Dropdown(
            id="variable",
            options=[{"label": i, "value": i} for i in df.columns if i not in ['Latitude', 'Longitude']],
            value=df.columns[0]
        ),
        # Repeat this pattern for other variables
    ], style={"width": "50%", "margin": "auto"}),
    dcc.Graph(id="city-graph"),
    dash_table.DataTable(id='table')
])

# Define callback to update the graph based on user input
@app.callback(
    [Output("city-graph", "figure"), Output("table", "data"), Output("table", "columns")],
    [Input("Median Rent", "value"),
     Input("Personal Income", "value"),
     Input("Median Sale Price","value"),
     Input("Walkable","value"),
     Input("Unemployment","value"),
     Input("Purchasing Power","value"),
     Input("Crime Rate","value"),
     Input("variable", "value")])
    # Add more Inputs for other variables
def update_graph(importance_personal_income, importance_median_rent,importance_median_sale_price,importance_walkable,importance_unemployment,importance_purchasing_power,importance_crime, selected_variable):
    # Weighted sum of variables
    df["Weighted_Sum"] = (df["Median.Rent"] * importance_personal_income +
                          df["personal.income"] * importance_median_rent + df["median.sale.price"]*importance_median_sale_price +df["Walkable"]*importance_walkable+df["Unemployment"]*importance_unemployment+
                          df["Purchase_power_100_dollars"]*importance_purchasing_power+df["Violent_Crime_Rate_per_100k"]*importance_crime)
    # Apply power transformation for better visualization
    df["Weighted_Sum"] = df["Weighted_Sum"] ** 3    
    
    # Define color scale mapping each political affiliation to a specific color
    color_scale = {
        "Democrat": "blue",
        "Republican": "red",
        "Independent": "yellow",
        "Nonpartisan": "grey"
    }
    
    # Plotly scatter plot
    #fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', scope='usa', size="Weighted_Sum", color="Political_Affiliation",
    #                     color_discrete_map=color_scale, custom_data=["REGION", "Weighted_Sum"])
    fig = px.scatter_geo(df, lat='Latitude', lon='Longitude', scope='usa', size="Weighted_Sum", color=selected_variable,
                     color_discrete_map=color_scale, custom_data=["REGION", "Weighted_Sum"])
    
    fig.update_layout(title="Best Metro Areas by Personal Livability Score",
                      mapbox_style="carto-positron",
                      mapbox_zoom=3,
                      mapbox_center={"lat": 37.0902, "lon": -95.7129})
    
    fig.update_traces(hovertemplate='<b>%{customdata[0]}</b><br>Weighted Sum: %{customdata[1]:.2f}')
    
    sorted_df = df.sort_values("Weighted_Sum", ascending=False)
    table_data = sorted_df[["REGION", "Weighted_Sum"]].to_dict('records')
    table_columns = [{"name": i, "id": i} for i in sorted_df[["REGION", "Weighted_Sum"]].columns]
    
    return fig, table_data, table_columns
# Run the app
if __name__ == "__main__":
    app.run_server(debug=True,port=8052)

In [ ]:
#| warning: false
#| code-fold: true
df = pd.read_csv("../final_data/final_data.csv")

In [ ]:
#| warning: false
#| code-fold: true
# #def east/west
def east_west(region):
    west_coast = ['Seattle', 'San Francisco', 'Los Angeles', 'San Diego', 'Portland']
    east_coast = ['Boston', 'New York', 'Washington, DC', 'Miami', 'Philadelphia', 'Atlanta', 'Baltimore', 'Virginia Beach', 'Charlotte', 'Tampa', 'Orlando', 'Miami']
    if region in west_coast:
        return "west"
    elif region in east_coast:
        return 'east'
    else:
        return 'cent'
    
df["coast"] = df["REGION"].apply(east_west)

# Sort the DataFrame by 'Year_Fahrenheit' column
# df_sorted = df.sort_values(by='Year_Fahrenheit')


#make df for each part of the country
#east
df_east = df[df['coast'] == 'east']
df_east = df_east.sort_values(by='Year_Fahrenheit')
#west
df_west = df[df['coast'] == 'west']
df_west = df_west.sort_values(by='Year_Fahrenheit')
#cent
df_cent = df[df['coast'] == 'cent']
df_cent = df_cent.sort_values(by='Year_Fahrenheit')

# For average temperature in west region
fig1 = px.bar(df_west, x='REGION', y='Year_Fahrenheit', color='Political_Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Western Metro Areas')

# For average temperature in central region
fig2 = px.bar(df_cent, x='REGION', y='Year_Fahrenheit', color='Political_Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Central Metro Areas')
# For average temperature in east region
fig3 = px.bar(df_east, x='REGION', y='Year_Fahrenheit', color='Political_Affiliation', 
              color_discrete_map=color_scale, title='Average Temperature in Eastern Metro Areas')

# Create subplots with 1 row and 3 columns
fig = make_subplots(rows=1, cols=3, subplot_titles=('West Coast Metro Areas', 'Central Metro Areas', 'East Coast Metro Areas'))

# Add the plots to the subplots
fig.add_trace(fig1['data'][0], row=1, col=1)
fig.add_trace(fig2['data'][0], row=1, col=2)
fig.add_trace(fig3['data'][0], row=1, col=3)

# Update layout
fig.update_layout(title_text='Average Temperature by Metro Area', yaxis_title='Temperature (F)', showlegend=False)
# Update bar width
fig.update_traces(width=.75)
# Show the plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#walkability

#sort
df = df.sort_values(by='Walkable', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='Walkable', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Walkability Score', title='Walkability Score by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#walkability
#sort
df = df.sort_values(by='Purchase_power_100_dollars', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='Purchase_power_100_dollars', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Purchasing Power by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#median rent
#sort
df = df.sort_values(by='Median.Rent', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='Median.Rent', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Median Rent by Metro Region')

# Hide legend
fig.update_traces(showlegend=False)
#plot
fig.show()

In [ ]:
#| warning: false
#| code-fold: true
#median.sale.price
#sort
df = df.sort_values(by='median.sale.price', ascending=False)
#plot
fig = px.bar(df, x='REGION', y='median.sale.price', color='Political_Affiliation', color_discrete_map=color_scale)

#Update layout
fig.update_layout(xaxis_title='Metro Region', yaxis_title='Purchase Power (per $100)', title='Median Home Sale Price by Metro Area')

#Hide legend
fig.update_traces(showlegend=False)
fig.show()